In [21]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Load data
df = pd.read_csv('data_halodoc_unordered.csv', delimiter=';')

# Create doc IDs starting from 1
df['doc_id'] = range(1, len(df) + 1)

print(f"Total drugs in dataset: {len(df)}")
print(f"Columns: {list(df.columns)}")
df.head()


Total drugs in dataset: 4732
Columns: ['name', 'price', 'Aturan Pakai', 'Dosis', 'Efek Samping', 'Golongan Produk', 'Indikasi Umum', 'Kemasan', 'Komposisi', 'Kontra Indikasi', 'Manufaktur', 'No. Registrasi', 'Perhatian', 'uses', 'doc_id']


,name,price,Aturan Pakai,Dosis,Efek Samping,Golongan Produk,Indikasi Umum,Kemasan,Komposisi,Kontra Indikasi,Manufaktur,No. Registrasi,Perhatian,uses,doc_id
0,Sildenafil Citrate 50 mg 4 Tablet,Rp96.500 - Rp168.500,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet",Sildenafil citrate 50 mg,Hipersensitivitas. Pasien yang menggunakan nit...,Generic Manufacturer,BPOM: GKL1433531717A1* *) Obat ini merupakan o...,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,SILDENAFIL CITRATE merupakan obat yang digunak...,1
1,Viagra 50 mg 1 Tablet,Rp156.800 - Rp164.900,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Blister @ 4 Tablet Salut Selaput",Sildenafil citrate 50 mg,Pasien yang menggunakan nitrat organik intermi...,Pfizer,BPOM: DKI1690401417A1,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,VIAGRA merupakan obat yang digunakan untuk men...,2
2,Ericfil Tablet 100 mg,Rp77.100 - Rp104.000,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, muka merah, dispepsia, hidung te...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet salut selaput",Sildenafil citrate 100 mg,"Pasien yang menggunakan nitrat organik, baik s...",Novell Pharmaceutical Laboratories,BPOM: DKL1333528917A1,HARUS DENGAN RESEP DOKTER. Pria yang tidak dia...,ERICFIL merupakan obat yang digunakan untuk me...,3
3,Cialis Tablet 10 mg,Rp197.300 - Rp203.500,Diberikan sebelum atau setelah makan,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, pusing, dispepsia, nyeri punggun...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 2 Blister @ 2 Tablet Salut Selaput",Tadalafil 10 mg,"Pemberian bersama dengan nitrat oksida, nitrat...",NaN,NaN,NaN,CIALIS 10 MG merupakan obat dengan kandungan T...,4
4,Ericfil 50 mg Odf,Rp36.500 - Rp44.400,Cara penggunaan Ericfil ODF: 1. Buka kemasan d...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 4 Strip @ 1 ODF",Sildenafil Citrate 50 mg,Hipersensitif terhadap komponen VIASTAR BLUE 5...,Novell Pharmaceutical Laboratories,BPOM: DKL2233554519A1,Hati-hati penggunaan obat in pada pasien denga...,ERICFIL 50 MG ODF merupakan obat yang digunaka...,5


In [22]:
# Define queries
query_pendek = ['obat jantung', 'obat stres', 'krim wajah']
query_panjang = [
    'obat oles untuk luka bakar ringan yang tidak perih',
    'obat herbal apa yang sudah terdaftar BPOM',
    'obat flu untuk balita'
]

print("Short queries:", query_pendek)
print("Long queries:", query_panjang)


Short queries: ['obat jantung', 'obat stres', 'krim wajah']
Long queries: ['obat oles untuk luka bakar ringan yang tidak perih', 'obat herbal apa yang sudah terdaftar BPOM', 'obat flu untuk balita']


In [23]:
# Text preprocessing function
def preprocess_text(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()
    # Remove special characters but keep spaces
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('indonesian') + stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

# Preprocess query
def preprocess_query(query):
    return preprocess_text(query)

print("Preprocessing function defined")


Preprocessing function defined


In [24]:
# Function to search for short queries (using uses and indikasi umum only)
def search_short_query(query, df, top_k=10):
    """
    Search for short queries using only 'uses' and 'Indikasi Umum' columns
    """
    # Combine uses and indikasi umum columns
    search_text = df['uses'].fillna('') + ' ' + df['Indikasi Umum'].fillna('')
    
    # Preprocess the combined text
    processed_texts = search_text.apply(preprocess_text)
    processed_query = preprocess_query(query)
    
    # Create TF-IDF vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_texts)
    query_vector = vectorizer.transform([processed_query])
    
    # Calculate cosine similarity
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Get top results
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        if similarities[idx] > 0:  # Only include results with similarity > 0
            results.append({
                'doc_id': df.iloc[idx]['doc_id'],
                'name': df.iloc[idx]['name'],
                'similarity': similarities[idx],
                'uses': df.iloc[idx]['uses'],
                'indikasi_umum': df.iloc[idx]['Indikasi Umum']
            })
    
    return results

print("Short query search function defined")


Short query search function defined


In [25]:
# Function to search for long queries (using additional relevant columns)
def search_long_query(query, df, top_k=10):
    """
    Search for long queries using uses, indikasi umum, and additional relevant columns
    """
    # Determine additional columns based on query content
    additional_cols = []
    
    if 'luka bakar' in query.lower() or 'oles' in query.lower():
        additional_cols.extend(['Aturan Pakai', 'Dosis'])
    elif 'herbal' in query.lower() or 'bpom' in query.lower():
        additional_cols.extend(['No. Registrasi', 'Golongan Produk'])
    elif 'flu' in query.lower() or 'balita' in query.lower():
        additional_cols.extend(['Dosis', 'Kontra Indikasi'])
    
    # Combine relevant columns
    search_text = df['uses'].fillna('') + ' ' + df['Indikasi Umum'].fillna('')
    for col in additional_cols:
        if col in df.columns:
            search_text += ' ' + df[col].fillna('')
    
    # Preprocess the combined text
    processed_texts = search_text.apply(preprocess_text)
    processed_query = preprocess_query(query)
    
    # Create TF-IDF vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_texts)
    query_vector = vectorizer.transform([processed_query])
    
    # Calculate cosine similarity
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Get top results
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        if similarities[idx] > 0:  # Only include results with similarity > 0
            result = {
                'doc_id': df.iloc[idx]['doc_id'],
                'name': df.iloc[idx]['name'],
                'similarity': similarities[idx],
                'uses': df.iloc[idx]['uses'],
                'indikasi_umum': df.iloc[idx]['Indikasi Umum']
            }
            # Add additional columns used in search
            for col in additional_cols:
                if col in df.columns:
                    result[col.lower().replace(' ', '_')] = df.iloc[idx][col]
            results.append(result)
    
    return results

print("Long query search function defined")


Long query search function defined


In [26]:
# Generate groundtruth for short queries
print("=== GROUNDTRUTH UNTUK QUERY PENDEK ===")
print("Menggunakan kolom: uses + Indikasi Umum")
print()

short_query_results = {}

for query in query_pendek:
    print(f"Query: '{query}'")
    results = search_short_query(query, df, top_k=50)
    short_query_results[query] = results
    
    print(f"Top {len(results)} hasil:")
    for i, result in enumerate(results, 1):
        print(f"  {i}. Doc ID: {result['doc_id']}, Nama: {result['name']}")
        print(f"     Similarity: {result['similarity']:.4f}")
        print(f"     Uses: {result['uses'][:100]}..." if len(str(result['uses'])) > 100 else f"     Uses: {result['uses']}")
        print()
    print("-" * 80)
    print()


=== GROUNDTRUTH UNTUK QUERY PENDEK ===
Menggunakan kolom: uses + Indikasi Umum

Query: 'obat jantung'


Top 50 hasil:
  1. Doc ID: 4251, Nama: Nu Q Ten Com 10 Kapsul
     Similarity: 0.3753
     Uses: NU Q TEN COM 10 KAPSUL merupakan suplemen yang mengandung Q10 Q10 digunakan sebagai katalis alami un...

  2. Doc ID: 4255, Nama: Nutriwell Co Q-10 30 Kapsul
     Similarity: 0.2385
     Uses: Nutriwell Co Q10 merupakan suplemen yang mengandung Nano Coenzyme Q10 (Ubidecarenone) 60 mg, Soybean...

  3. Doc ID: 4252, Nama: Nutrimax Coenzyme Q10 30 Tablet
     Similarity: 0.2287
     Uses: NUTRIMAX COENZYME Q10 berperan penting dalam produksi energi, serta berperan sebagai antioksidan Mem...

  4. Doc ID: 4248, Nama: Natural Factors Coenzyme Q10 30 Kapsul
     Similarity: 0.2232
     Uses: NATURAL FACTORS COENZYME Q10 100 MG 30 KAPSUL merupakan suplemen makanan yang mengandung Coenzyme Q1...

  5. Doc ID: 481, Nama: Combivent UDV 10 Vial
     Similarity: 0.1806
     Uses: COMBIVENT UDV merupakan obat asma yang mengandung Ipratropium bromide dan Salbutamol sulphate Salbut...

  6. Doc ID: 4358,

In [27]:
# Generate groundtruth for long queries
print("=== GROUNDTRUTH UNTUK QUERY PANJANG ===")
print("Menggunakan kolom: uses + Indikasi Umum + kolom relevan tambahan")
print()

long_query_results = {}

for query in query_panjang:
    print(f"Query: '{query}'")
    results = search_long_query(query, df, top_k=100)
    long_query_results[query] = results
    
    print(f"Top {len(results)} hasil:")
    for i, result in enumerate(results, 1):
        print(f"  {i}. Doc ID: {result['doc_id']}, Nama: {result['name']}")
        print(f"     Similarity: {result['similarity']:.4f}")
        print(f"     Uses: {result['uses'][:100]}..." if len(str(result['uses'])) > 100 else f"     Uses: {result['uses']}")
        
        # Show additional columns used
        additional_info = []
        for key, value in result.items():
            if key not in ['doc_id', 'name', 'similarity', 'uses', 'indikasi_umum'] and value:
                additional_info.append(f"{key}: {str(value)[:50]}...")
        
        if additional_info:
            print(f"     Additional: {', '.join(additional_info)}")
        print()
    print("-" * 80)
    print()


=== GROUNDTRUTH UNTUK QUERY PANJANG ===
Menggunakan kolom: uses + Indikasi Umum + kolom relevan tambahan

Query: 'obat oles untuk luka bakar ringan yang tidak perih'
Top 50 hasil:
  1. Doc ID: 2633, Nama: Lanakeloid 10 Kaplet
     Similarity: 0.4414
     Uses: LANAKELOID merupakan obat dengan kandungan Centella Asiatica dalam bentuk kaplet Obat ini digunakan ...
     Additional: aturan_pakai: Dikonsumsi bersamaan dengan makanan atau tidak..., dosis: 3 kali sehari 1 kaplet....

  2. Doc ID: 2469, Nama: Bioplacenton Gel 15 g
     Similarity: 0.3807
     Uses: BIOPLACENTON merupakan obat yang mengandung Placenta Extract dan Neomycin sulfate Obat ini biasa dig...
     Additional: aturan_pakai: Dioleskan pada kulit yang sakit/ lepuh/ terbakar s..., dosis: PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJUK D...

  3. Doc ID: 2604, Nama: Lanakeloid-E Cream 10 g
     Similarity: 0.3744
     Uses: LANAKELOID-E CREAM merupakan obat dengan kandungan Centella Asiatica Phytosome, Vit E dalma bentuk k

In [28]:
# Create final groundtruth summary
print("=== RINGKASAN GROUNDTRUTH FINAL ===")
print()

# Combine all results
all_results = {}

# Add short query results
for query, results in short_query_results.items():
    all_results[query] = {
        'type': 'short',
        'columns_used': ['uses', 'Indikasi Umum'],
        'results': [(r['doc_id'], r['name']) for r in results]
    }

# Add long query results
for query, results in long_query_results.items():
    additional_cols = []
    if 'luka bakar' in query.lower() or 'oles' in query.lower():
        additional_cols = ['Aturan Pakai', 'Dosis']
    elif 'herbal' in query.lower() or 'bpom' in query.lower():
        additional_cols = ['No. Registrasi', 'Golongan Produk']
    elif 'flu' in query.lower() or 'balita' in query.lower():
        additional_cols = ['Dosis', 'Kontra Indikasi']
    
    all_results[query] = {
        'type': 'long',
        'columns_used': ['uses', 'Indikasi Umum'] + additional_cols,
        'results': [(r['doc_id'], r['name']) for r in results]
    }

# Display summary
for query, info in all_results.items():
    print(f"Query: '{query}'")
    print(f"Tipe: {info['type']}")
    print(f"Kolom yang digunakan: {', '.join(info['columns_used'])}")
    print(f"Hasil ({len(info['results'])} obat):")
    for doc_id, name in info['results']:
        print(f"  - Doc ID: {doc_id}, Nama: {name}")
    print()


=== RINGKASAN GROUNDTRUTH FINAL ===

Query: 'obat jantung'
Tipe: short
Kolom yang digunakan: uses, Indikasi Umum
Hasil (50 obat):
  - Doc ID: 4251, Nama: Nu Q Ten Com 10 Kapsul
  - Doc ID: 4255, Nama: Nutriwell Co Q-10 30 Kapsul
  - Doc ID: 4252, Nama: Nutrimax Coenzyme Q10 30 Tablet
  - Doc ID: 4248, Nama: Natural Factors Coenzyme Q10 30 Kapsul
  - Doc ID: 481, Nama: Combivent UDV 10 Vial
  - Doc ID: 4358, Nama: Folas 400 mcg 10 Tablet
  - Doc ID: 4536, Nama: Erha Nutraceuticals Skincore 30 Kapsul
  - Doc ID: 4713, Nama: Wellness Slimming Formula 60 Kapsul
  - Doc ID: 3906, Nama: Beneuron 10 Tablet
  - Doc ID: 3916, Nama: Mega We Care Nat B 30 SoftGel
  - Doc ID: 1384, Nama: Duspatalin 135 mg 10 Tablet
  - Doc ID: 4529, Nama: Wellness Natural Vitamin E 400 IU 60 Softgel
  - Doc ID: 3944, Nama: Wellness Natural Vitamin E 400 IU 60 Softgel
  - Doc ID: 4528, Nama: Wellness Natural Vitamin E 400 I.U 150 Softgel
  - Doc ID: 3943, Nama: Wellness Natural Vitamin E 400 I.U 150 Softgel
  - Doc

In [29]:
# Save groundtruth to CSV for easy access
import csv

# Create groundtruth CSV
groundtruth_data = []

for query, info in all_results.items():
    for doc_id, name in info['results']:
        groundtruth_data.append({
            'query': query,
            'query_type': info['type'],
            'columns_used': '; '.join(info['columns_used']),
            'doc_id': doc_id,
            'drug_name': name
        })

# Save to CSV
groundtruth_df = pd.DataFrame(groundtruth_data)
groundtruth_df.to_csv('groundtruth_results.csv', index=False)

print("Groundtruth telah disimpan ke 'groundtruth_results.csv'")
print(f"Total {len(groundtruth_data)} hasil groundtruth")
print()
print("Preview groundtruth CSV:")
print(groundtruth_df.head(10))


Groundtruth telah disimpan ke 'groundtruth_results.csv'
Total 300 hasil groundtruth

Preview groundtruth CSV:
          query query_type         columns_used  doc_id  \
0  obat jantung      short  uses; Indikasi Umum    4251   
1  obat jantung      short  uses; Indikasi Umum    4255   
2  obat jantung      short  uses; Indikasi Umum    4252   
3  obat jantung      short  uses; Indikasi Umum    4248   
4  obat jantung      short  uses; Indikasi Umum     481   
5  obat jantung      short  uses; Indikasi Umum    4358   
6  obat jantung      short  uses; Indikasi Umum    4536   
7  obat jantung      short  uses; Indikasi Umum    4713   
8  obat jantung      short  uses; Indikasi Umum    3906   
9  obat jantung      short  uses; Indikasi Umum    3916   

                                drug_name  
0                  Nu Q Ten Com 10 Kapsul  
1             Nutriwell Co Q-10 30 Kapsul  
2         Nutrimax Coenzyme Q10 30 Tablet  
3  Natural Factors Coenzyme Q10 30 Kapsul  
4                   